<a href="https://colab.research.google.com/github/YuanchengJin/CP1404/blob/master/Faceswap_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructions

Faceswap - Google Colab

You also need Faceswap GUI for this to work, since you can't extract faces here, yet.

Files you need to upload on Google Drive
1.   face_a.zip (source face pics you want to swap)
2.   face_b.zip (faces you want to get swapped)

> Zipped files with extracted faces and alignments.fsa

3.   train.ini
> Your train.ini from your local system, in Windows this is in: C:\Users\YourName\faceswap\config

Here is what filesystem looks like in Google Drive
*   /colab_files/faceswap/faces/face_a.zip
*   /colab_files/faceswap/faces/face_b.zip
*   /colab_files/faceswap/config/train.ini

Recommended folders to create on Google Drive:
*   /colab_files/faceswap/models/YourModelName
*   /colab_files/faceswap/output/timelapse

Here is what filesystem should look like in this Google Colab after the "Mount Google Drive" step below.
*   /content/drive/My Drive/colab_files/faceswap/faces/face_a.zip
*   /content/drive/My Drive/colab_files/faceswap/faces/face_b.zip
*   /content/drive/My Drive/colab_files/faceswap/config/train.ini

# Usage Notes

This is my personal SOP for handling the Faceswap process. I find it is best to tweak your settings, extract, test your training, and convert everything using your local system.

But training on my laptop took weeks to get the same results that you can get from just days by using a Google Colab.

*   Extract, Align, then Zip all Faces A
*   Extract, Align, then Zip all Faces B
*   Modify the "Run Training" script below to your preferences
*   Train in this Colab Notebook
*   Watch Google Drive timestamp folder to see how the training is improving
*   Download YourModelName to your local system
*   Convert using the downloaded model


# Keep Session Alive
 (it may not work due to latest Google Colab Captcha changes.)



In [1]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button");
     btn.click()
     }

   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect");
     btn.click()
     }
  }

setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


# Check your GPU
If it isn't **Tesla T4** or **Tesla P100**, go Runtime->Factory Reset Runtime until you get one.

In [2]:
!nvidia-smi

import tensorflow as tf
print("Tensorflow version: " + tf.__version__)

Sun Jun 29 09:09:56 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   51C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Setup Faceswap

In [3]:
#@title Set Time Zone
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Turkey /etc/localtime
!date

#above is for HST, you can find yours in
#/usr/share/zoneinfo


Sun Jun 29 12:10:03 PM +03 2025


In [5]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
#@title Get your training data(aka the faces you extracted in app)
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_a.zip" .
!cp "/content/drive/My Drive/colab_files/faceswap/faces/face_b.zip" .

!unzip face_a.zip -d face_a
!unzip face_b.zip -d face_b

!rm face_a.zip
!rm face_b.zip


cp: cannot stat '/content/drive/My Drive/colab_files/faceswap/faces/face_a.zip': No such file or directory
cp: cannot stat '/content/drive/My Drive/colab_files/faceswap/faces/face_b.zip': No such file or directory
unzip:  cannot find or open face_a.zip, face_a.zip.zip or face_a.zip.ZIP.
unzip:  cannot find or open face_b.zip, face_b.zip.zip or face_b.zip.ZIP.
rm: cannot remove 'face_a.zip': No such file or directory
rm: cannot remove 'face_b.zip': No such file or directory


In [8]:
#@title Get the latest Faceswap
!git clone https://github.com/deepfakes/faceswap.git

Cloning into 'faceswap'...
remote: Enumerating objects: 14694, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 14694 (delta 23), reused 19 (delta 19), pack-reused 14657 (from 3)
Receiving objects: 100% (14694/14694), 198.72 MiB | 33.97 MiB/s, done.
Resolving deltas: 100% (10485/10485), done.


In [9]:
#@title Copy your configuration(train.ini) file
!cp "/content/drive/My Drive/colab_files/faceswap/config/train.ini" faceswap/config/
!ls -lA faceswap/config/
!cat faceswap/config/train.ini

cp: cannot stat '/content/drive/My Drive/colab_files/faceswap/config/train.ini': No such file or directory
total 0
-rw-r--r-- 1 root root 0 Jun 29 12:11 .keep
cat: faceswap/config/train.ini: No such file or directory


In [11]:
#@title Install Tensorflow

!pip install -r faceswap/requirements/requirements_nvidia.txt

Ignoring pywin32: markers 'sys_platform == "win32"' don't match your environment
Ignoring pynvx: markers 'sys_platform == "darwin"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
ERROR: Ignored the following versions that require a different python version: 1.21.2 Requires-Python >=3.7,<3.11; 1.21.3 Requires-Python >=3.7,<3.11; 1.21.4 Requires-Python >=3.7,<3.11; 1.21.5 Requires-Python >=3.7,<3.11; 1.21.6 Requires-Python >=3.7,<3.11
ERROR: Could not find a version that satisfies the requirement tensorflow<2.11.0,>=2.10.0 (from versions: 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0)
ERROR: No matching distribution found for tensorflow<2.11.0,>=2.10.0


In [12]:
!python3 faceswap/tools.py mask -a 'face_b/alignments.fsa' -i 'face_b' -it faces -M vgg-clear -p all -o '/drive/MyDrive/colab_files/faceswap/faces/facexb/mask' -b 3 -t 4 -ot mask -L INFO


First time configuration. Please select the required backend
1: CPU, 2: DIRECTML, 3: NVIDIA, 4: APPLE SILICON, 5: ROCM: 3
Faceswap config written to: /content/faceswap/config/.faceswap
Setting Faceswap backend to NVIDIA
usage: tools.py [-h] {alignments,effmpeg,manual,mask,model,preview,sort} ...

positional arguments:
  {alignments,effmpeg,manual,mask,model,preview,sort}
    alignments          This command lets you perform various tasks pertaining
                        to an alignments file.
    effmpeg             This command allows you to easily execute common
                        ffmpeg tasks.
    manual              This command lets you perform various actions on
                        frames, faces and alignments files using visual tools.
    mask                This tool allows you to generate, import, export or
                        preview masks for existing alignments.
    model               This tool lets you perform actions on saved Faceswap
                     

# Run Training
Iteration number is up to you but it's recommended that you should do until the number you are okay with. You should check your timelapse folder for that.

And if my calculations are right, trainer is doing nearly 360 iterations in 10 mins, which means 2160 iterations in 1 hr.

In [13]:
#set variables start
num_iterations = "100000"
save_every = "360"
save_model_every = "25000"
batch_num = "16"
num_gpus = "1"

trainer_type = "villain"

model_dir = "/content/drive/My Drive/colab_files/faceswap/models/YourModelName"
alignments_file_a = "face_a/2_alignments.fsa"
alignments_file_b = "face_b/alignments.fsa"
timelapse_dir = "/content/drive/My Drive/colab_files/faceswap/output/timelapse"
#set variables end

!python3 faceswap/faceswap.py train \
  -A 'face_a' \
  -B 'face_b'  \
  -m '{model_dir}' \
  -t '{trainer_type}' \
  -bs '{batch_num}' \
  -it '{num_iterations}' \
  -s '{save_every}' \
  -ss '{save_model_every}' \
  -tia 'face_a' \
  -tib 'face_b' \
  -to '{timelapse_dir}'


Setting Faceswap backend to NVIDIA
06/29/2025 12:12:39 INFO     Log level set to: INFO
E0000 00:00:1751188359.486154    1650 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751188359.491978    1650 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
DEBUG:2025-06-29 12:12:41,160:jax._src.path:31: etils.epath found. Using etils.epath for file I/O.
06/29/2025 12:12:43 ERROR    The maximum supported Tensorflow is version (2, 10) but you have version (2, 18) installed. Please downgrade Tensorflow.


# Convert

In [14]:
!python3 faceswap/faceswap.py convert -i '/content/drive/My Drive/colab_files/faceswap/Editor.avi' -o '/content/drive/My Drive/colab_files/faceswap/Results' -al '/content/drive/My Drive/colab_files/faceswap/2_alignments.fsa' -m '/content/drive/My Drive/colab_files/faceswap/models/YourModelName' -c match-hist -M none -w ffmpeg -osc 100 -l 0.4 -j 0 -t realface -L INFO

Setting Faceswap backend to NVIDIA
usage: faceswap.py [-h] {extract,train,convert,gui} ...

positional arguments:
  {extract,train,convert,gui}
    extract             Extract the faces from pictures or a video
    train               Train a model for the two faces A and B
    convert             Convert source pictures or video to a new one with the
                        face swapped
    gui                 Launch the Faceswap Graphical User Interface

options:
  -h, --help            show this help message and exit
faceswap.py: error: unrecognized arguments: -t realface
